# Урок 9. Часть 3

1. Арендуем компьютер с ГПУ в облаке и настраиваем его.
1. Запускаем API сервер с инференсом LLM (деплоим LLM).
1. Запускаем веб интерфейс для чата с запущенной LLM (показываем LLM).

## Аренда компьютера

https://my.selectel.ru/
https://docs.selectel.ru/cloud/servers/manage/connect-to-server/

![](gpu/1.png)
*****
![](gpu/2.png)
*****
![](gpu/3.png)
*****
![](gpu/4.png)
*****
![](gpu/5.png)
*****
![](gpu/6.png)
*****
![](gpu/7.png)
*****
![](gpu/8.png)
*****
![](gpu/9.png)
*****

## Деплой LLM
```bash
# Устанавливаем необходимые пакеты
apt install git unzip

# Создаем новое виртуальное окружение для нашего инференса LLM
conda create -n llmenv python=3.10 -y
conda activate llmenv

# Устанавливаем vllm для CUDA 11.8
export VLLM_VERSION=0.4.2
export PYTHON_VERSION=310
pip install https://github.com/vllm-project/vllm/releases/download/v${VLLM_VERSION}/vllm-${VLLM_VERSION}+cu118-cp${PYTHON_VERSION}-cp${PYTHON_VERSION}-manylinux1_x86_64.whl --extra-index-url https://download.pytorch.org/whl/cu118
pip install flash_attn

# Запускаем инференс LLM
python -m vllm.entrypoints.openai.api_server --model solidrust/Hermes-2-Pro-Llama-3-8B-AWQ --dtype auto --api-key token-abc123 --served-model-name llm
```

- Ссылка на модель: https://huggingface.co/solidrust/Hermes-2-Pro-Llama-3-8B-AWQ
- Ссылка на VLLM: https://docs.vllm.ai/en/stable/, https://github.com/vllm-project/vllm
- Ссылка на VLLM документацию сервера: https://docs.vllm.ai/en/latest/serving/openai_compatible_server.html#command-line-arguments-for-the-server

## Деплой LLM Web Interface
https://github.com/reflex-dev/reflex-chat

```bash

# Создаем новое виртуальное окружение для чатбота
conda create -n chatbotenv python=3.10 -y
conda activate chatbotenv

# Устанавливаем нужные библиотеки
pip install reflex

# Создаем папку для проекта и переходим туда
mkdir chat
cd chat

# Инициализируем проект с веб интерфейсом чатилкой (https://github.com/reflex-dev/reflex-chat)
reflex init --template chat --loglevel debug

# Доустанавливаем нужные библиотеки
pip install -r requirements.txt
```

Редактируем код, чтобы он ходил в локальный LLM сервер, а не в chatgpt:

```bash
cd chat
nano state.py
```

```python
# Добавляем строчку после импортов
client = OpenAI(base_url="http://0.0.0.0:8000/v1")

# В функции openai_process_question меняем код
# Было
session = OpenAI().chat.completions.create(
# Стало
session = client.chat.completions.create(
```

```bash
# Запускаем веб интерфейс
export OPENAI_API_KEY="token-abc123"
export OPENAI_MODEL="llm"
reflex run --loglevel debug
```

Открываем в браузере `http://SERVER_URL:3000/`

### Дальнейшие шаги
- Сделать настройку системного промпта и параметров генерации
- Добавить удаление/редактирование сообщений, копирование текста
- Переписать код целиком чтобы лучше разобраться в том как все устроено, улучшить дизайн, оптимизировать запросы
- Добавить заранее заготовленные промпты для пользователей
- Добавить использование инструментов LLM (агент)
- ...


*****
- Упаковать все в Docker
- Забрать все переменные (адрес LLM сервера, ключ от API, название LLM) в конфиг
- Попробовать другой фреймворк для запуска LLM сервера: https://github.com/huggingface/text-generation-inference, https://huggingface.co/docs/text-generation-inference/index

## Резюме

1. Арендовали компьютер с ГПУ в облаке
2. Запустили на компьютере API сервер с инференсом LLM
3. Запустили на компьютере веб интерфейс для чата с LLM
4. Получили заготовку для пет проекта